# Read necessary packages

In [ ]:
import os
import sys
import yaml

import numpy as np
import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

from lasso import Project

from lasso import bicounty

import pickle

%load_ext autoreload
%autoreload 2

import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

# Specify Input Directories

In [ ]:
pickle_dir = "../../" #Set to directory where the pickle files are stored
card_dir = "../../ProjectCards"#Set to directory where the cards are stored
transit_dir = "../../Transit"
lasso_dir = '..'

In [ ]:
from lasso import Parameters
parameters = Parameters(lasso_base_dir = lasso_dir)

# Read Scenario Pickle File

#### There are two pickle files available: one for 2015 and one for 2020. To create the 2020 network:
##### Read the 2015 pickle file, apply the 2020 projects.

#### To create the 2035 network, you can:
##### - Read the 2015 Pickle file, apply the 2020 and 2035 project cards or
##### - Read the 2020 Pickle file and apply the 2035 project cards only.

In [ ]:
pickle_file_name = os.path.join(pickle_dir, "scenario_2015.pickle")
curr_scenario = pickle.load(open(pickle_file_name, 'rb'))

#### Some of the link names are in a list format, they need to be updated

In [ ]:
curr_scenario.road_net.links_df['name']=curr_scenario.road_net.links_df['name'].apply(lambda x: "" if type(x)==int else x)
curr_scenario.road_net.links_df['name']=curr_scenario.road_net.links_df['name'].apply(lambda x: util.shorten_name(x))

# Create 2020 Network
### (Skip to list of projects for the 2035 scenario if you are starting from 2020 scenario)

## List of Project Cards to be Applied to the 2015 Scenario

In [ ]:
project_card_list = []

scenario_2015_cards = ['Added_networkforexternals_Sacto2_GN_11142022.yml',
                        'assignable_and_ace_transit_connectors.yml',
                        'CCTA.yml',
                        'Externals_SJQ_Add_complete_network_10032022.yml',
                        'Sonoma_Marin_Area_Rail_Transit_add_rail_link.yml',
                        'Year 2015 Modify Express Lanes and USE on I-580 EB (segment 02) Hacienda Drive to Airway Blvd_v2.yml',
                        'Year 2015 Modify HOV Lanes and USE on CA-237 EB US 101 to North 1st Street_v2.yml',
                        'Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101_v2.yml',
                        'Year 2015 Modify HOV Lanes and USE on I-680 NB from Alcosta Blvd to Livorna Road_v2.yml',
                        'Year 2015 Modify HOV Lanes and USE on I-680 SB from Benicia Bridge to Alcosta Blvd_v2.yml',
                        'year_2015_managed_lane_i680n_ca_242_to_benicia_bridge.yml']

for card in scenario_2015_cards:
    _filename = os.path.join(card_dir, '2020',card)
    card = ProjectCard.read(_filename, validate = False)
    project_card_list.append(card)

In [ ]:
for item in project_card_list:
    curr_scenario.apply_project(item)

In [ ]:
all_line_links_2020=pd.read_csv(os.path.join(transit_dir, "2020", "all_transit_links.csv"))
all_line_links_2020.tail()

In [ ]:
model_2020_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = curr_scenario.road_net, 
    parameters = parameters
)

In [ ]:
updated_2020_net = mtc.roadway_standard_to_mtc_network(model_2020_net, parameters=parameters)

In [ ]:
outpur_network_2020_dir = os.path.join(pickle_dir, '2020')
os.makedirs(outpur_network_2020_dir, exist_ok=True)

In [ ]:
updated_2020_net.links_mtc_df=updated_2020_net.links_mtc_df[updated_2020_net.links_mtc_df['geometry'].notnull()]

In [ ]:
all_line_links_2020['has_transit'] = 1
has_transit = pd.merge(all_line_links_2020, updated_2020_net.links_mtc_df[['A','B','model_link_id']])
updated_2020_net.links_mtc_df['name']=updated_2020_net.links_mtc_df['name'].apply(lambda x: "" if type(x)==int else x)
updated_2020_net.links_mtc_df['name']=updated_2020_net.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))
updated_2020_net.links_mtc_df['has_transit']=updated_2020_net.links_mtc_df['model_link_id'].map(dict(zip(has_transit['model_link_id'],has_transit['has_transit']))).fillna(0)

In [ ]:
model_2020_net.links_mtc_df = updated_2020_net.links_mtc_df.copy()
model_2020_net.nodes_mtc_df = updated_2020_net.nodes_mtc_df.copy()

In [ ]:
model_2020_net.links_mtc_df['BRT'] = 0

In [ ]:
model_2020_net.write_roadway_as_fixedwidth(
    output_dir = outpur_network_2020_dir,
    output_link_txt = 'links.txt',
    output_node_txt = 'nodes.txt',
    output_link_header_width_txt = 'links_header_width.txt',
    output_node_header_width_txt = 'nodes_header_width.txt',
    output_cube_network_script = 'make_complete_network_from_fixed_width_file.s',
    #drive_only = True
)

# List of Project Cards for 2035 Scenario 
## (skip here if you are starting from 2020 scenario)
These projects add new links/nodes to the network. 

In [ ]:
project_card_list = []
set1_card_2035 = [
                    ### Projects in San Joaqun
                    'san_joaquin_05_sj07_1006.yml',
                    'san_joaquin_06_sj07_1005.yml',
                    'san_joaquin_07_sj11_1001.yml',
                    'san_joaquin_08_sj18_1001.yml',
                    'san_joaquin_09_sj07_1014.yml',
                    'san_joaquin_01_sj14_1001.yml',
                    'san_joaquin_02_sj14_1002.yml',
                    'san_joaquin_03_sj07_1003.yml',
                    'san_joaquin_04_sj07_1008.yml',
                    ### Projects modifying highway related to transit improvements (BRT lines, Station/Stop Updates etc.)
                    '0_01_CongestionPricingDowntownSF_21-T10-091.yml',
                    'Sonoma_Marin_Area_Rail_Transit_add_rail_link.yml', #If you are starting from 2015 pickle file, comment this card out since it is already added on the 2015 network.
                    '0_02_CongestionPricingTreasureIsland_21-T10-092.yml',
                    '0_06_SR-237 Corridor Improvement 21-T06-043.wrangler',
                    '0_08_I-680 Corridor Improvement 21-T06-022.wrangler',
                    '0_10_1_21-T06-027-FBP_SM_027_US101_92.yml',
                    '0_10_2_21-T06-027-FBP_SM_033_US101_Holly_Interchange_add.yml',
                    '0_10_3_21-T06-027-FBP_SM_033_US101_Holly_Interchange_mod.yml',
                    '0_10_4_21-T06-027-FBP_SM_035_Peninsula_101_OnOffRamps.yml',
                    '0_10_5_21-T06-027-STIP_ProduceAve.yml',
                    '0_11_1_21-T06-028-FBP_SC_072_US101_Trimble_Interchange.yml',
                    '0_11_2_21-T06-028-FBP_SC_073_BlossomHill_101Wide.yml',
                    '0_11_5_21-T06-028-FBP_SC_081_US101_SR237.wrangler',
                    '0_11_7_21-T06-028-FBP_SC_083_US101_Zanker_Skyport_Interchange.yml',
                    '0_12_21-T06-015-MAJ_SOL070020_I80_I680_SR12_Int_2B_7_mod.yml',
                    '10_21-T11-114_Transform_Valley_Link_add_rail_link.yml',
                    '2_1_14_MAJ_MTC050027_Berkeley_Ferry_add_ferry_link.yml',
                    '2_4_15_21-T11-97-MAJ_MissionBay_SF_Ferry_add_ferry_link.yml',
                    '2_5_16_21_T11-98-MAJ_RedwoodCity_SF_Ferry_add_ferry_link.yml',
                    '3_21-T10-083-Central_Subway_ToChinaTown_add_rail_link.yml',
                    '5_1_Rex_Blue_add_highway_link.yml',
                    'BART_2020_Fremont_to_Berryessa_add_rail_link.yml',
                    'BART_2036_Berryessa_to_Santa_Clara_add_rail_link.yml',
                    'Caltrain_Peninsula_Corridor_Electrification_Expansion_add_rail_link.yml',
                    'extra_brt_14th_st_mission_blvd_brt1.yml',
                    'extra_brt_albany_to_oakland_brt2.yml',
                    'extra_brt_broadway_brt1.yml',
                    'extra_brt_grand_ave_brt1.yml',
                    'extra_brt_lower_dimond_to_alameda_brt2.yml',
                    'extra_brt_marin_ave_brt2.yml',
                    'extra_brt_merritt_to_ashland_brt2.yml',
                    'extra_brt_mission_grain_to_grimmer_brt2.yml',
                    'extra_brt_oakland_brt2.yml',
                    'extra_brt_ralph_appezzato_memorial_pwky_brt1.yml',
                    'extra_brt_thomas_berkley_way_grand_ave_brt1.yml',
                    'year_2020_better_market_st_eb.yml',
                    'year_2020_better_market_st_wb.yml',
                    'year_2020_broadway_brt.yml',
                    'year_2030_i_80_wb_bus_only_hov_extension.yml',
                    'year_2020_richmond_san_rafael_bridge_access_improvement.yml',
                    'year_2020_tempo_brt.yml',
                    'year_2021_geary_brt_phase1_eb.yml',
                    'year_2021_geary_brt_phase1_wb.yml',
                    'year_2021_sr4_wb_added_gp_lane_sr242_to_i680.yml',
                    'year_2022_van_ness_brt_nb.yml',
                    'year_2022_van_ness_brt_sb.yml',
                    ### Adding new express lanes
                    'year_2023_us101_el_whipple_to_i380.yml',
                    'year_2024_san_pablo_ave_rapid_corridors_project.yml',
                    'year_2025_bay_bridge_forward_i580wb_hov_extension.yml',
                    'year_2025_i680_sr237_to_sr_84_hov_lanes_new_construction.yml',
                    'year_2025_i80_solano_managed_lanes.yml',
                    'year_2025_sr84_widening_rubyhill_to_i680.yml',
                    'year_2025_us101_marin_sonoma_narrows.yml',
                    'year_2028_geary_brt_phase2_eb.yml',
                    'year_2028_geary_brt_phase2_wb.yml',
                    'year_2029_san_pablo_ave_bus_bike_project.yml',
                    'year_2030_23rd_st_brt.yml',
                    'year_2030_broadway_streetscape_improvement_project.yml',
                    'year_2030_i280_us101_nb_hov.yml',
                    'year_2030_i280_us101_sb_hov.yml',
                    'year_2030_i580_wb_rsr_forward.yml',
                    'year_2030_i680_express_lanes_new_roadway_nb_sb.yml',
                    'year_2030_i680_nb_express_lane_completion_2.yml',
                    'year_2030_i680_nb_pttl.yml',
                    'year_2030_i680_sb_pttl.yml',
                    'year_2030_sr37_sears_point_to_mareisland_hov.yml',
                    'year_2030_sr4_wb_el_sr242_to_i680.yml',
                    'year_2030_us101_ml_north_of_i380_to_sf_sm_countyline.yml',
                    'year_2033_i680_el_calaveras_to_us101.yml',
                    'year_2035_bike_walk_access_on_bridges.yml'
                    ]

for card in set1_card_2035:
    _filename = os.path.join(card_dir, '2035','1-2',card)
    card = ProjectCard.read(_filename, validate = False)
    project_card_list.append(card)


# Apply project Cards (Phase 1)



In [ ]:
for item in project_card_list:
    curr_scenario.apply_project(item)

## Create Model Network Object

In [ ]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = curr_scenario.road_net, 
    parameters = parameters
)


## Convert the model network to MTC specific format
This steps creates all the parallel links, adds new nodes and links and calculates the link attributes

In [ ]:
updated_net = mtc.roadway_standard_to_mtc_network(model_net, parameters=parameters)

## One link in the network has a null geometry. Deleting it and adding it back again with a diferent project card

In [ ]:
updated_net.links_mtc_df=updated_net.links_mtc_df[updated_net.links_mtc_df['geometry'].notnull()]

# Add has_transit attribute to links carrying transit

In [ ]:
transit_file = os.path.join(transit_dir, "2035", "transit.lin")
# gather network links by parsing through the node sequences in transit.lin
with open(transit_file, "r") as f:
    lines = f.readlines()

    all_line_links = pd.DataFrame()
    curr_line = None
    line_node_seq = None

    for txt in lines:
        if txt.startswith("LINE NAME="):
            # print (txt.split("=")[1])
            # store the current line name
            curr_line = txt.split("\"")[1]
            # reset line_node_seq as an empty list
            line_node_seq = []

        # add to node sequence if the first item of txt after split by "," and remove whitespace is digit
        # if txt.strip().split(",")[0].replace(" ", "").replace("-", "").isdigit():
        if txt.strip().split(",")[0].replace("-", "").isdigit():
            node = int(txt.strip().split(",")[0].replace("-", ""))
            line_node_seq.append(node)

        if curr_line and txt == "\n":
            print(f"processed line: {curr_line}")
            # convert previous line_node_seq into df
            line_links = pd.DataFrame({"line": curr_line, "A": line_node_seq[:-1], "B": line_node_seq[1:]})
            # add to all_line_links
            all_line_links = pd.concat([all_line_links, line_links]).reset_index(drop=True)

all_line_links["A"] = all_line_links["A"].astype(int)
all_line_links["B"] = all_line_links["B"].astype(int)
all_line_links=all_line_links.drop_duplicates(['A','B'])

In [ ]:
# transit_links=pd.read_csv(os.path.join(card_dir,'2035', "all_transit_links.csv"))
transit_all_line_linkslinks=all_line_links.rename(columns={'line':'has_transit'})
all_line_links['has_transit'] = 1
has_transit = pd.merge(all_line_links, updated_net.links_mtc_df[['A','B','model_link_id']])

In [ ]:
updated_net.links_mtc_df['name']=updated_net.links_mtc_df['name'].apply(lambda x: "" if type(x)==int else x)
updated_net.links_mtc_df['name']=updated_net.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))
updated_net.links_mtc_df['has_transit']=updated_net.links_mtc_df['model_link_id'].map(dict(zip(has_transit['model_link_id'],has_transit['has_transit']))).fillna(0)

In [ ]:
curr_scenario.road_net.links_df = updated_net.links_mtc_df.copy()
curr_scenario.road_net.nodes_df = updated_net.nodes_mtc_df.copy()

In [ ]:
v01_scenario = Scenario.create_scenario(base_scenario = curr_scenario)

In [ ]:
v01_scenario.road_net.nodes_df['model_node_id'] = v01_scenario.road_net.nodes_df['N']

In [ ]:
project_card_list = []
set_2_2035 = [  #This one updates one link that was deleted in a previous step because of null geometry
                'add_extra_transit_lane.yml',
                #Remaining cards were created from cube log files, so they modify existing links and their attributes
                'dublin_blvd_nparkway_extension.yml',
                'year_2021_i880_nb_el_dixon_landing_to_leweeling_blvd_hot_conversion.yml',
                'year_2021_i880_sb_el_hegenberger_rd_to_dixon_landing_hot_conversion.yml',
                'year_2025_i80_solano_managed_lanes_phase2.yml',
                'year_2025_scl101_el_sr237_to_i880.yml',
                'year_2025_sr85_el_i280_to_sr87_eb.yml',
                'year_2025_sr85_el_i280_to_sr87_wb.yml',
                'year_2025_sr85_el_sr87_us101_wb.yml',
                'year_2025_sr85_el_us101_to_i280_eb.yml',
                'year_2025_sr85_el_us101_to_i280_wb.yml',
                'year_2025_us101_el.yml',
                'year_2027_i880_el_sr237_to_us101.yml',
                'year_2030_el_sr4_hillcrestave_to_ca242.yml',
                'year_2030_i680_nb_express_lane_completion_1.yml',
                'year_2030_i80_carquinez_to_baybridge_el.yml',
                'year_2030_sr85_el_i280_to_sr87_eb.yml',
                'year_2030_sr85_el_i280_to_sr87_wb.yml',
                'year_2030_sr85_el_i280_to_sr87_wb_v2.yml',
                'year_2030_sr87_el_us101_to_sr85.yml',
                'year_2035_bus_on_shoulder_strategy.yml',
                'year_2030_us101_el_whipple_to_i380_missing.yml',
                'year_2035_scl101_el_i880_to_eastdunneave.yml']

for card in set_2_2035:
    _filename = os.path.join(card_dir, '2035','3',card)
    card = ProjectCard.read(_filename, validate = False)
    project_card_list.append(card)

In [ ]:
for item in project_card_list:
    v01_scenario.apply_project(item)

In [ ]:
v01_scenario.road_net.links_df['model_link_id'] = np.where((v01_scenario.road_net.links_df['A']==1511000)&(v01_scenario.road_net.links_df['B']==6011000),
                                                            v01_scenario.road_net.links_df['model_link_id'].max()+1,
                                                            v01_scenario.road_net.links_df['model_link_id'])

v01_scenario.road_net.links_df['model_link_id'] = np.where((v01_scenario.road_net.links_df['A']==3031452)&(v01_scenario.road_net.links_df['B']==3078980),
                                                            v01_scenario.road_net.links_df['model_link_id'].max()+1,
                                                            v01_scenario.road_net.links_df['model_link_id'])

In [ ]:
model_net_updated = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v01_scenario.road_net, 
    parameters = parameters
)

In [ ]:
model_net_updated.nodes_mtc_df = model_net_updated.nodes_df.copy()
model_net_updated.links_mtc_df = model_net_updated.links_df.copy()

# Export Network

#### Export as Shapefile

In [ ]:
outpur_network_2035_dir = os.path.join(pickle_dir, '2035')
os.makedirs(outpur_network_2035_dir, exist_ok=True)

In [ ]:
model_net_updated.links_df[[ 'drive_access','name', 'roadway','walk_access', 
       'county', 'model_link_id', 'A', 'B', 'rail_only',
        'ft',
       'assignable', 'transit', 'nmt2010',
       'nmt2020', 'tollbooth', 'bus_only',  'managed',
        'tollseg', 'cntype', 'useclass_AM', 'useclass_EA',
       'useclass_EV', 'useclass_MD', 'useclass_PM', 'BRT', 'distance',
       'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_EV','geometry', 'has_transit']].to_file(os.path.join(outpur_network_2035_dir,"complete_network_2035.shp"))

#### Export as Cube Network

In [ ]:
model_net_updated.write_roadway_as_fixedwidth(
    output_dir = outpur_network_2035_dir,
    output_link_txt = 'links.txt',
    output_node_txt = 'nodes.txt',
    output_link_header_width_txt = 'links_header_width.txt',
    output_node_header_width_txt = 'nodes_header_width.txt',
    output_cube_network_script = 'make_complete_network_from_fixed_width_file.s',
    #drive_only = True
)